In [2]:
%load_ext autoreload
%autoreload 2

import feedparser as fp
import newspaper
import helpers
from newspaper import Source, Article

In [3]:
%%time
all_cnn_papers = helpers.compile_papers(rss_feeds_dict=helpers.CNN_LINKS_DICT)
all_fox_papers = helpers.compile_papers(rss_feeds_dict=helpers.FOX_LINKS_DICT)

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_latest.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_health.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_allpolitics.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_world.rss
[Parse lxml 

In [57]:
%%time
helpers.art_multithread_downloads(papers=all_cnn_papers, threads_per_source=2)

CPU times: user 12.3 s, sys: 2.2 s, total: 14.5 s
Wall time: 6min 43s


In [58]:
%%time
# Parse all articles:
for paper in all_cnn_papers:
    for article in paper.articles:
        article.parse()

/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag))


CPU times: user 4min 25s, sys: 6.56 s, total: 4min 31s
Wall time: 6min 34s


In [53]:
all_cnn_papers[2].set_description()

AttributeError: 'NoneType' object has no attribute 'cssselect'